In [55]:
import sys
!conda install --yes --prefix {sys.prefix} selenium
# !{sys.executable} -m pip install requests_html

In [56]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
from datetime import date

#import dload
# downloading packages

In [243]:
# set variables
today = date.today().strftime("%Y-%m-%d")
wework = 'https://www.wework.com'

In [247]:
driver = webdriver.Chrome('./chromedriver')
driver.get("https://www.wework.com/search?slug=new-york-city--NY&capacity=1") 
time.sleep(5)

req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')

driver.quit()
#soup

In [248]:
building_links = []
buildings = soup.find("div",{"id":"results"}).find_all('a')
for building in buildings:
    building_links.append(wework+building['href'])
building_links

['https://www.wework.com/buildings/980-6th-avenue--new-york-city--NY',
 'https://www.wework.com/buildings/16-e-34th-st--new-york-city--NY',
 'https://www.wework.com/buildings/430-park-avenue--new-york-city--NY',
 'https://www.wework.com/buildings/gotham-center--new-york-city--NY',
 'https://www.wework.com/buildings/8-w-126th-st--new-york-city--NY',
 'https://www.wework.com/buildings/140-crosby-st--new-york-city--NY',
 'https://www.wework.com/buildings/368-9th-ave--new-york-city--NY',
 'https://www.wework.com/buildings/880-3rd-avenue--new-york-city--NY',
 'https://www.wework.com/buildings/450-lexington-ave--new-york-city--NY',
 'https://www.wework.com/buildings/199-water-st--new-york-city--NY',
 'https://www.wework.com/buildings/33-east-33rd-street--new-york-city--NY',
 'https://www.wework.com/buildings/505-park-avenue--new-york-city--NY',
 'https://www.wework.com/buildings/7-west-18th-street--new-york-city--NY',
 'https://www.wework.com/buildings/10-east-40th-street--new-york-city--NY'

In [249]:
capacity_list = [num for num in range(1,21)]
capacity_extra = ['21-100','100%2B']
capacity_list

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [213]:
import random
from concurrent.futures import ThreadPoolExecutor
import threading

csv_writer_lock = threading.Lock()
options = webdriver.ChromeOptions()
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")


space_urls = dict()
def get_inventory(url):
    capacity_list = [str(num) for num in range(1,21)]+['21-100','100%2B']
    for capacity in capacity_list:
        url = url +'/reserve?capacity='+str(capacity)+'&start_date='+today
        driver = webdriver.Chrome(executable_path="chromedriver", options=options)
        driver.get(url) #define the url of the site you want to scrape
        req = driver.page_source
        soup0 = BeautifulSoup(req, 'html.parser')
        workspaces = soup0.find('div',{'class':'product-card'}).find_all('a',{'class':'selection-card'})
        workspaces.pop()

        for space in workspaces:
            key = space['href']
            cap = space.find('div',{'class':'ray-tag ray-tag--with-icon capacity-tag'}).text
            space_urls[key] = cap
    
#     with csv_writer_lock:
#         with open("inventory.csv", mode="a") as f1:
#             inv_writer = csv.writer(f1, delimiter=",")
#             for r in space_urls:
#                 inv_writer.writerow(r)
    return space_urls

def set_up_threads(urls, core):
    with ThreadPoolExecutor(max_workers=core) as executor:
        return executor.map(get_inventory, urls, timeout = 60)

set_up_threads(building_links, 5)

<generator object Executor.map.<locals>.result_iterator at 0x7fee310b53c0>

In [214]:
space_urls

{}

In [103]:
space_links = dict()

driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)
#wait = WebDriverWait(driver, 10)

for building in building_links:
    for capacity in capacity_list:
        link = building +'/reserve?capacity='+str(capacity)+'&start_date='+today
        driver.get(link) 
        #driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
        #wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="uc-btn-accept-banner"]'))).click()

        req = driver.page_source
        soup4 = BeautifulSoup(req, 'html.parser')

        workspaces = soup4.find('div',{'class':'product-card'}).find_all('a',{'class':'selection-card'})
        workspaces.pop()
        for space in workspaces:
            #space_links.append(wework+space['href'])
            key = space['href']#.split('/')[-1]
            cap = space.find('div',{'class':'ray-tag ray-tag--with-icon capacity-tag'}).text
            space_links[key] = cap
            if len(cap) > 0 and int(cap) > capacity:
                capacity = int(cap)
    for capacity in capacity_extra:
        link = building +'/reserve?capacity='+capacity+'&start_date='+today
        driver.get(link) 
        #driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
        #wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="uc-btn-accept-banner"]'))).click()

        req = driver.page_source
        soup4 = BeautifulSoup(req, 'html.parser')

        workspaces = soup4.find('div',{'class':'product-card'}).find_all('a',{'class':'selection-card'})
        workspaces.pop()
        for space in workspaces:
            #space_links.append(wework+space['href'])
            key = space['href']#.split('/')[-1]
            cap = space.find('div',{'class':'ray-tag ray-tag--with-icon capacity-tag'}).text
            space_links[key] = cap
        
driver.quit()
space_links

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: not connected to DevTools
  (Session info: chrome=91.0.4472.114)


In [104]:
space_links

{'/buildings/980-6th-avenue--new-york-city--NY/reserve/936b4506-e681-4af1-91b9-c359270fdf60': '2',
 '/buildings/980-6th-avenue--new-york-city--NY/reserve/a49501fb-e2f9-403d-8479-965dd5def54e': '2',
 '#': '',
 '/buildings/980-6th-avenue--new-york-city--NY/reserve/6cf8b30c-9775-4707-9ec2-72e866699d30': '7',
 '/buildings/980-6th-avenue--new-york-city--NY/reserve/e7991168-4adf-4f99-84cb-284b5b95c547': '7',
 '/buildings/980-6th-avenue--new-york-city--NY/reserve/6eec4f73-27a6-46c3-83fa-41a7d495009c': '10',
 '/buildings/980-6th-avenue--new-york-city--NY/reserve/c468abe5-79af-414e-ad21-fd0d3db79234': '8',
 '/buildings/980-6th-avenue--new-york-city--NY/reserve/adda2473-73fd-47a6-b0c9-5858a20bf14f': '8',
 '/buildings/980-6th-avenue--new-york-city--NY/reserve/ae58d870-4905-4f52-8b88-e8f457e46701': '14',
 '/buildings/980-6th-avenue--new-york-city--NY/reserve/f6111953-61a3-46cf-aca4-637c0e563321': '20',
 '/buildings/16-e-34th-st--new-york-city--NY/reserve/9375696c-f573-460d-994f-ef60259bdc01': '5',

In [151]:
import pandas as pd
import numpy as np

In [245]:
new_space_links = space_links.copy()
new_space_links.update(space_links1)
new_space_links.update(space_links2)
new_space_links.update(space_links3)
new_space_links.update(space_links4)

In [246]:
import string
inventory = pd.DataFrame.from_dict(new_space_links,orient='index').reset_index().rename(columns={'index':'address', 0:'capacity'})
inventory = inventory[inventory.capacity!='']
inventory.reset_index(drop=True, inplace=True)
inventory['url'] = wework+inventory['address'].astype(str)
inventory[['building', 'space']] = inventory['address'].str.split('/reserve/', 2, expand=True)
inventory[['building', 'city', 'state']] = inventory['building'].str.split('--', 2, expand=True)
inventory['building'] = inventory['building'].str.replace('/buildings/','').str.replace('-',' ').apply(lambda x: string.capwords(x))
inventory['city'] = inventory['city'].str.replace('-',' ').str.title()
inventory['state'] = inventory['state'].astype(str).str[:2]
inventory = inventory[['state','city','building','space','capacity','url']]
inventory.to_csv('wework_nyc_inventory.csv')
inventory

,state,city,building,space,capacity,url
0,NY,New York City,980 6th Avenue,936b4506-e681-4af1-91b9-c359270fdf60,2,https://www.wework.com/buildings/980-6th-avenu...
1,NY,New York City,980 6th Avenue,a49501fb-e2f9-403d-8479-965dd5def54e,2,https://www.wework.com/buildings/980-6th-avenu...
2,NY,New York City,980 6th Avenue,6cf8b30c-9775-4707-9ec2-72e866699d30,7,https://www.wework.com/buildings/980-6th-avenu...
3,NY,New York City,980 6th Avenue,e7991168-4adf-4f99-84cb-284b5b95c547,7,https://www.wework.com/buildings/980-6th-avenu...
4,NY,New York City,980 6th Avenue,6eec4f73-27a6-46c3-83fa-41a7d495009c,10,https://www.wework.com/buildings/980-6th-avenu...
...,...,...,...,...,...,...
589,NY,New York City,1460 Broadway,3c900ab7-a5e8-482c-9fe9-95b4d7acb71b,14,https://www.wework.com/buildings/1460-broadway...
590,NY,New York City,1460 Broadway,3373fce2-0e7c-45cc-b536-debaa2cc9eb6,15,https://www.wework.com/buildings/1460-broadway...
591,NY,New York City,1460 Broadway,698c1f8a-6250-4157-b285-eb05195f2fee,17,https://www.wework.com/buildings/1460-broadway...
592,NY,New York City,1460 Broadway,6746f857-ec12-4a87-92cf-6401e2edc788,17,https://www.wework.com/buildings/1460-broadway...


In [68]:
# import requests

# url = 'https://www.wework.com/buildings/880-3rd-avenue--new-york-city--NY'
# page = requests.get(url)
# soup1 = BeautifulSoup(page.text, 'html.parser')
# reserve_link = soup1.find("div",{"class":"product-cards-block"}).find('a')['href']
# reserve_link

In [41]:
# link = url+'/reserve?capacity=2&start_date='+today
# page = requests.get(link)
# soup2 = BeautifulSoup(page.text, 'html.parser')
# jsvar = str(soup2.find_all('script')[0]).strip('<script>\n      window.clientSideInjectionVars = ').strip(';\n    </')

# import json
# res = json.loads(jsvar)
# res['marketBuildings']

In [52]:
# from requests_html import AsyncHTMLSession
# asession = AsyncHTMLSession()

# r = asession.get(link)
# await r.html.arender()

In [164]:
# driver = webdriver.Chrome('./chromedriver')
# driver.implicitly_wait(10)
# wait = WebDriverWait(driver, 10)

# capacity=1
# space_links = dict()

# link = exp +'/reserve?capacity='+str(capacity)+'&start_date='+today
# driver.get(link) 
# #driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
# #wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="uc-btn-accept-banner"]'))).click()

# req = driver.page_source
# soup4 = BeautifulSoup(req, 'html.parser')

# workspaces = soup4.find('div',{'class':'product-card'}).find_all('a',{'class':'selection-card'})
# workspaces.pop()
# for space in workspaces:
#     key = space['href']#.split('/')[-1]
#     cap = space.find('div',{'class':'ray-tag ray-tag--with-icon capacity-tag'}).text
#     space_links[key] = cap
#     if len(cap.text)>0 and int(cap) > capacity:
#         capacity = int(cap)

# driver.quit()
# space_links

In [169]:
space_links1 = dict()

driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(5)
#wait = WebDriverWait(driver, 10)

for building in building_links[24:]:
    for capacity in capacity_list:
        link = building +'/reserve?capacity='+str(capacity)+'&start_date='+today
        driver.get(link) 
        #driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
        #wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="uc-btn-accept-banner"]'))).click()

        req = driver.page_source
        soup4 = BeautifulSoup(req, 'html.parser')

        workspaces = soup4.find('div',{'class':'product-card'}).find_all('a',{'class':'selection-card'})
        workspaces.pop()
        for space in workspaces:
            #space_links.append(wework+space['href'])
            key = space['href']#.split('/')[-1]
            cap = space.find('div',{'class':'ray-tag ray-tag--with-icon capacity-tag'}).text
            space_links1[key] = cap
            if len(cap) > 0 and int(cap) > capacity:
                capacity = int(cap)
    for capacity in capacity_extra:
        link = building +'/reserve?capacity='+capacity+'&start_date='+today
        driver.get(link) 
        #driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
        #wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="uc-btn-accept-banner"]'))).click()

        req = driver.page_source
        soup4 = BeautifulSoup(req, 'html.parser')

        workspaces = soup4.find('div',{'class':'product-card'}).find_all('a',{'class':'selection-card'})
        workspaces.pop()
        for space in workspaces:
            #space_links.append(wework+space['href'])
            key = space['href']#.split('/')[-1]
            cap = space.find('div',{'class':'ray-tag ray-tag--with-icon capacity-tag'}).text
            space_links1[key] = cap
        
driver.quit()
space_links1

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: not connected to DevTools
  (Session info: chrome=91.0.4472.114)


In [215]:
space_links1

{'#': '',
 '/buildings/511-w-25th-st--new-york-city--NY/reserve/ea83fb69-6aa0-45fd-ac0f-a1042d1e3731': '2',
 '/buildings/511-w-25th-st--new-york-city--NY/reserve/9819f98f-ad1e-4340-b6cb-aa52c26bb5d7': '1',
 '/buildings/511-w-25th-st--new-york-city--NY/reserve/fb25a986-0215-4da6-8155-db9c8c581761': '3',
 '/buildings/511-w-25th-st--new-york-city--NY/reserve/e7532635-a726-46d5-bd26-88c8e049bd91': '4',
 '/buildings/511-w-25th-st--new-york-city--NY/reserve/f6f5367d-2f2a-4a04-a49b-520d43331ec2': '6',
 '/buildings/511-w-25th-st--new-york-city--NY/reserve/33261f9a-e8eb-4ffd-bb78-b51fedba4128': '6',
 '/buildings/511-w-25th-st--new-york-city--NY/reserve/f6f80578-dffe-4d6d-af6c-14e84de1281a': '11',
 '/buildings/511-w-25th-st--new-york-city--NY/reserve/30b9cf42-6220-4344-80ec-930ec73fb238': '12',
 '/buildings/511-w-25th-st--new-york-city--NY/reserve/3065b65a-e886-4957-aba3-6bfea74c2fe5': '15',
 '/buildings/511-w-25th-st--new-york-city--NY/reserve/29c7c2dd-6f89-4a59-9761-caea4dedc6df': '15',
 '/bui

In [171]:
building_links2 = ['https://www.wework.com/buildings/135-e-57th-st--new-york-city--NY',
 'https://www.wework.com/buildings/3537-36th-st--new-york-city--NY',
 'https://www.wework.com/buildings/12-e-49th-st--new-york-city--NY',
 'https://www.wework.com/buildings/11-park-pl--new-york-city--NY',
 'https://www.wework.com/buildings/311-w-43rd-st--new-york-city--NY',
 'https://www.wework.com/buildings/71-5th-avenue--new-york-city--NY',
 'https://www.wework.com/buildings/125-w-25th-st--new-york-city--NY',
 'https://www.wework.com/buildings/142-w-57th-st--new-york-city--NY',
 'https://www.wework.com/buildings/33-irving-pl--new-york-city--NY',
 'https://www.wework.com/buildings/379-w-broadway--new-york-city--NY',
 'https://www.wework.com/buildings/195-montague-st--new-york-city--NY',
 'https://www.wework.com/buildings/110-wall-st--new-york-city--NY',
 'https://www.wework.com/buildings/1460-broadway--new-york-city--NY']

space_links2 = dict()

driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(5)
#wait = WebDriverWait(driver, 10)

for building in building_links2:
    for capacity in capacity_list:
        link = building +'/reserve?capacity='+str(capacity)+'&start_date='+today
        driver.get(link) 
        #driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
        #wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="uc-btn-accept-banner"]'))).click()

        req = driver.page_source
        soup4 = BeautifulSoup(req, 'html.parser')

        workspaces = soup4.find('div',{'class':'product-card'}).find_all('a',{'class':'selection-card'})
        workspaces.pop()
        for space in workspaces:
            #space_links.append(wework+space['href'])
            key = space['href']#.split('/')[-1]
            cap = space.find('div',{'class':'ray-tag ray-tag--with-icon capacity-tag'}).text
            space_links2[key] = cap
            if len(cap) > 0 and int(cap) > capacity:
                capacity = int(cap)
    for capacity in capacity_extra:
        link = building +'/reserve?capacity='+capacity+'&start_date='+today
        driver.get(link) 
        #driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
        #wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="uc-btn-accept-banner"]'))).click()

        req = driver.page_source
        soup4 = BeautifulSoup(req, 'html.parser')

        workspaces = soup4.find('div',{'class':'product-card'}).find_all('a',{'class':'selection-card'})
        workspaces.pop()
        for space in workspaces:
            #space_links.append(wework+space['href'])
            key = space['href']#.split('/')[-1]
            cap = space.find('div',{'class':'ray-tag ray-tag--with-icon capacity-tag'}).text
            space_links2[key] = cap
        
driver.quit()
space_links2

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=91.0.4472.114)


In [216]:
space_links2

{'/buildings/135-e-57th-st--new-york-city--NY/reserve/e2f1b9aa-80c2-49ed-8edd-c101d86203e1': '1',
 '/buildings/135-e-57th-st--new-york-city--NY/reserve/e2850bc0-4cb6-42de-9a1d-98a1c0b61c85': '2',
 '/buildings/135-e-57th-st--new-york-city--NY/reserve/fcb7a4c9-cb1c-45cf-9195-a413caa682e9': '1',
 '/buildings/135-e-57th-st--new-york-city--NY/reserve/f946f5e9-c691-487b-a9df-c8b5a246498c': '3',
 '/buildings/135-e-57th-st--new-york-city--NY/reserve/100450e5-eba3-47b6-8803-227df633ff99': '3',
 '/buildings/135-e-57th-st--new-york-city--NY/reserve/fbefdfa5-db9f-485d-8428-1b370adf9ca2': '4',
 '/buildings/135-e-57th-st--new-york-city--NY/reserve/6fe136f9-ba09-44cd-967c-e4470be3000a': '3',
 '/buildings/135-e-57th-st--new-york-city--NY/reserve/95a9395f-3a03-4673-b3b4-1ddc921f7efc': '4',
 '/buildings/135-e-57th-st--new-york-city--NY/reserve/d2acaea3-44dd-4aca-8a4d-32d0bd291e11': '4',
 '/buildings/135-e-57th-st--new-york-city--NY/reserve/c73fd69f-9d4f-4143-9a7a-f5cfa23c4cb3': '4',
 '/buildings/135-e-5

In [223]:
building_links3 = ['https://www.wework.com/buildings/205-hudson-st--new-york-city--NY',
 'https://www.wework.com/buildings/27-e-28th-st--new-york-city--NY',
 'https://www.wework.com/buildings/222-broadway--new-york-city--NY',
 'https://www.wework.com/buildings/85-broad-st--new-york-city--NY',
 'https://www.wework.com/buildings/dumbo-heights-77-sands--new-york-city--NY',
 'https://www.wework.com/buildings/500-7th-ave--new-york-city--NY',
 'https://www.wework.com/buildings/54-w-40th-st--new-york-city--NY',
 'https://www.wework.com/buildings/524-broadway--new-york-city--NY',
 'https://www.wework.com/buildings/135-madison-ave--new-york-city--NY',
 'https://www.wework.com/buildings/18-west-18th-street--new-york-city--NY',
 'https://www.wework.com/buildings/8-west-40th-street--new-york-city--NY',
 'https://www.wework.com/buildings/30-west-21st-street--new-york-city--NY',
 'https://www.wework.com/buildings/53-beach-st--new-york-city--NY',
 'https://www.wework.com/buildings/38-west-21st-street--new-york-city--NY',
 'https://www.wework.com/buildings/149-5th-avenue--new-york-city--NY']

space_links3 = dict()

driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(5)
#wait = WebDriverWait(driver, 10)

for building in building_links3:
    for capacity in capacity_list:
        link = building +'/reserve?capacity='+str(capacity)+'&start_date='+today
        driver.get(link) 
        #driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
        #wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="uc-btn-accept-banner"]'))).click()

        req = driver.page_source
        soup4 = BeautifulSoup(req, 'html.parser')

        workspaces = soup4.find('div',{'class':'product-card'}).find_all('a',{'class':'selection-card'})
        workspaces.pop()
        for space in workspaces:
            #space_links.append(wework+space['href'])
            key = space['href']#.split('/')[-1]
            cap = space.find('div',{'class':'ray-tag ray-tag--with-icon capacity-tag'}).text
            space_links3[key] = cap
            if len(cap) > 0 and int(cap) > capacity:
                capacity = int(cap)
    for capacity in capacity_extra:
        link = building +'/reserve?capacity='+capacity+'&start_date='+today
        driver.get(link) 
        #driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
        #wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="uc-btn-accept-banner"]'))).click()

        req = driver.page_source
        soup4 = BeautifulSoup(req, 'html.parser')

        workspaces = soup4.find('div',{'class':'product-card'}).find_all('a',{'class':'selection-card'})
        workspaces.pop()
        for space in workspaces:
            #space_links.append(wework+space['href'])
            key = space['href']#.split('/')[-1]
            cap = space.find('div',{'class':'ray-tag ray-tag--with-icon capacity-tag'}).text
            space_links3[key] = cap
        
driver.quit()
space_links3

{'/buildings/205-hudson-st--new-york-city--NY/reserve/f5d187b8-91d2-4ff7-b203-3d9cd028678f': '1',
 '/buildings/205-hudson-st--new-york-city--NY/reserve/fd11e8bf-7e1c-4698-8799-c31322c8220f': '2',
 '/buildings/205-hudson-st--new-york-city--NY/reserve/d0649447-9d48-4c67-8543-ccda39bb979a': '3',
 '/buildings/205-hudson-st--new-york-city--NY/reserve/d34b5dc6-8509-4ab1-9ab1-16b3ecf76db2': '3',
 '/buildings/205-hudson-st--new-york-city--NY/reserve/79d5df30-3a8e-45c7-80d1-6569404711d1': '4',
 '/buildings/205-hudson-st--new-york-city--NY/reserve/b81b4888-18de-4556-8d9a-c120fc70d512': '4',
 '/buildings/205-hudson-st--new-york-city--NY/reserve/e196ecfb-9470-49ef-b551-cbd2c3500683': '5',
 '/buildings/205-hudson-st--new-york-city--NY/reserve/e2f557e7-19bf-4639-906d-ada4f2a3da6f': '6',
 '/buildings/205-hudson-st--new-york-city--NY/reserve/e5a1f0aa-0136-4d00-87eb-c72f96fc164d': '10',
 '/buildings/205-hudson-st--new-york-city--NY/reserve/c6d73728-ffa9-4f87-b9ba-445499b92ebf': '12',
 '/buildings/205-h

In [244]:
building_links4 = ['https://www.wework.com/buildings/185-madison-ave--new-york-city--NY',
                'https://www.wework.com/buildings/134-n-4th-st--new-york-city--NY',
                'https://www.wework.com/buildings/1460-broadway--new-york-city--NY']

space_links4 = dict()

driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(5)
#wait = WebDriverWait(driver, 10)

for building in building_links4:
    print(building)
    for capacity in capacity_list:
        link = building +'/reserve?capacity='+str(capacity)+'&start_date='+today
        driver.get(link) 
        #driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
        #wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="uc-btn-accept-banner"]'))).click()

        req = driver.page_source
        soup4 = BeautifulSoup(req, 'html.parser')

        workspaces = soup4.find('div',{'class':'product-card'}).find_all('a',{'class':'selection-card'})
        workspaces.pop()
        for space in workspaces:
            #space_links.append(wework+space['href'])
            key = space['href']#.split('/')[-1]
            cap = space.find('div',{'class':'ray-tag ray-tag--with-icon capacity-tag'}).text
            space_links4[key] = cap
            if len(cap) > 0 and int(cap) > capacity:
                capacity = int(cap)
    for capacity in capacity_extra:
        link = building +'/reserve?capacity='+capacity+'&start_date='+today
        driver.get(link) 
        #driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()
        #wait.until(EC.element_to_be_clickable((By.XPATH,'//*[@id="uc-btn-accept-banner"]'))).click()

        req = driver.page_source
        soup4 = BeautifulSoup(req, 'html.parser')

        workspaces = soup4.find('div',{'class':'product-card'}).find_all('a',{'class':'selection-card'})
        workspaces.pop()
        for space in workspaces:
            #space_links.append(wework+space['href'])
            key = space['href']#.split('/')[-1]
            cap = space.find('div',{'class':'ray-tag ray-tag--with-icon capacity-tag'}).text
            space_links4[key] = cap
        
driver.quit()
space_links4

https://www.wework.com/buildings/185-madison-ave--new-york-city--NY
https://www.wework.com/buildings/134-n-4th-st--new-york-city--NY
https://www.wework.com/buildings/1460-broadway--new-york-city--NY


{'#': '',
 '/buildings/134-n-4th-st--new-york-city--NY/reserve/d0cf43a9-469f-4840-8a1b-12cd6388507b': '1',
 '/buildings/134-n-4th-st--new-york-city--NY/reserve/6758032e-cdbf-4ba6-95fc-cb9637a1e30a': '1',
 '/buildings/134-n-4th-st--new-york-city--NY/reserve/752fd5d2-f216-441f-bf1b-84632c652ed1': '2',
 '/buildings/134-n-4th-st--new-york-city--NY/reserve/21d092d6-adb6-4e9c-b3c5-d93e15e42059': '1',
 '/buildings/134-n-4th-st--new-york-city--NY/reserve/3c335cf9-4316-4235-b904-de8ee2a9fa4c': '3',
 '/buildings/134-n-4th-st--new-york-city--NY/reserve/39875206-5e21-4c9f-9317-f5d5bf767a5a': '4',
 '/buildings/134-n-4th-st--new-york-city--NY/reserve/d6c2c661-9e70-4db7-8417-bd3446785325': '4',
 '/buildings/134-n-4th-st--new-york-city--NY/reserve/705a1837-09b7-40f0-9192-2ee64bf67e19': '6',
 '/buildings/134-n-4th-st--new-york-city--NY/reserve/128d856e-cc99-435d-9725-b75ce4123792': '8',
 '/buildings/134-n-4th-st--new-york-city--NY/reserve/557600c4-2e6f-45ba-a85a-17b0f3e5d3ea': '10',
 '/buildings/1460-b

In [224]:
space_links4

{'#': ''}

In [ ]:
['https://www.wework.com/buildings/10-east-38th-st--new-york-city--NY',
                'https://www.wework.com/buildings/1410-broadway--new-york-city--NY',
                'https://www.wework.com/buildings/83-maiden-lane--new-york-city--NY',
                'https://www.wework.com/buildings/546-5th-avenue--new-york-city--NY',
                'https://www.wework.com/buildings/525-broadway--new-york-city--NY',
                'https://www.wework.com/buildings/57-east-11th-street--new-york-city--NY',
                'https://www.wework.com/buildings/49-west-27th-street--new-york-city--NY',
                'https://www.wework.com/buildings/500-5th-avenue--new-york-city--NY',
                'https://www.wework.com/buildings/6-east-32nd-street--new-york-city--NY',
                'https://www.wework.com/buildings/1156-6th-avenue--new-york-city--NY',
                'https://www.wework.com/buildings/385-5th-avenue--new-york-city--NY',
                'https://www.wework.com/buildings/the-club-row-building--new-york-city--NY',
                'https://www.wework.com/buildings/183-madison-ave--new-york-city--NY',
                'https://www.wework.com/buildings/145-w-45th-street--new-york-city--NY']